# Model Calibration


In [ ]:
using Pkg
Pkg.activate(".")

In [ ]:
using CSV
using DataFrames
using DiffFusion
using PlotlyJS

## Reference Data

### Volatility Data

In [ ]:
file_name = "data/standard_deviation_30days.csv"
std_table = DataFrame(CSV.File(file_name))
std_table = stack(std_table, 3:size(std_table)[2])
std_table[!, "VOLATILITY"] = std_table[!, "value"] / sqrt(30.0/365.0);
std_table[!, "YEARS"] = std_table[!, "MONTHS"] / 12;

### Correlation Data

In [ ]:
file_name = "data/correlations_30days.csv"
corr_table = DataFrame(CSV.File(file_name))
corr_table = stack(corr_table, 5:size(corr_table)[2])
corr_table[!, "YEARS1"] = Int.(corr_table[!, "MONTHS1"] / 12)
corr_table[!, "YEARS2"] = Int.(corr_table[!, "MONTHS2"] / 12);

## Plotting Functions

### Plot Volatility

In [ ]:
function plot_volatility(
    std_table,
    currency,
    model_terms,
    model_values,
    scaling,
    )
    #
    table = filter(:CURRENCY => ==(currency), std_table)
    table[!,"VOLATILITY"] = table[!,"VOLATILITY"] * scaling # in percent/bp
    #
    b = box(table, x=:YEARS, y=:VOLATILITY, name="historical volatility")
    #
    s = scatter(
        x = model_terms,
        y = model_values * scaling,
        mode = "markers",
        marker = attr(size=12, line=attr(width=2, color="DarkSlateGrey")),
        name="model-implied volatility",
    )
    #
    if scaling == 100.0
        scaling_unit = " (%)"
    elseif scaling == 10000.0
        scaling_unit = " (bp)"
    else
        scaling_unit = ""
    end
    layout = Layout(
        title= currency * " Volatility",
        xaxis_title="rate term",
        yaxis_title="volatility" * scaling_unit,
        legend_title="Output",
        font=attr(
            family="Arial",
            size=12,
            color="RebeccaPurple"
        )
    )
    return plot([b, s], layout)
end

### Plot Correlations

In [ ]:
@inline function iuppert(k::Integer,n::Integer)
    i = n - 1 - floor(Int,sqrt(-8*k + 4*n*(n-1) + 1)/2 - 0.5)
    j = k + i + ( (n-i+1)*(n-i) - n*(n-1) )÷2
    return i, j
end

function make_index_list(curr_and_year::AbstractVector{<:Tuple})
    l = curr_and_year  # abbreviation
    n = length(curr_and_year)
    return [
        (l[idx[1]][1], l[idx[1]][2], l[idx[2]][1], l[idx[2]][2])
        for idx in (iuppert(k, n) for k in 1:Int(n*(n-1)/2))
    ]
end

function make_index_list(curr_and_year_1::AbstractVector{<:Tuple}, curr_and_year_2::AbstractVector{<:Tuple})
    l1 = curr_and_year_1  # abbreviation
    l2 = curr_and_year_2  # abbreviation
    return [
        (l1[1], l1[2], l2[1], l2[2])
        for l1 in curr_and_year_1 for l2 in curr_and_year_2
    ]
end

function make_sub_table(table, index_list)
    t = table  # abbreviation
    l = index_list  # abbreviation
    tmp = vcat([
        [
            t[(t.CURRENCY1 .== l[1]) .&& (t.YEARS1 .== l[2]) .&& (t.CURRENCY2 .== l[3]) .&& (t.YEARS2 .== l[4]),:],
            t[(t.CURRENCY1 .== l[3]) .&& (t.YEARS1 .== l[4]) .&& (t.CURRENCY2 .== l[1]) .&& (t.YEARS2 .== l[2]),:],
        ]
        for l in index_list
    ]...)
    return vcat(tmp...)
end

function plot_correlation(
    table,
    index_list,
    values,
    )
    #
    t = make_sub_table(table, index_list)
    t[!, "XLABEL"] = t[!, "CURRENCY1"] .* "_" .* string.(t[!, "YEARS1"]) .* "__" .* t[!, "CURRENCY2"] .* "_" .* string.(t[!, "YEARS2"])
    t[!, "CORRELATION"] = t[!, "value"] .* 100  # in percent
    b = box(t, x=:XLABEL, y=:CORRELATION, name="historical correlation")
    #
    xlabels = [
        t[1] * "_" * string.(t[2]) * "__" * t[3] * "_" .* string(t[4])
        for t in index_list
    ]
    yvalues = values .* 100 # in percent
    s = scatter(
        x = xlabels,
        y = yvalues,
        mode = "markers",
        marker = attr(size=12, line=attr(width=2, color="DarkSlateGrey")),
        name="model-implied correlation",
    )
    #
    layout = Layout(
        title= "Correlation",
        xaxis_title="risk factors",
        yaxis_title="correlation (%)",
        legend_title="Output",
        font=attr(
            family="Arial",
            size=12,
            color="RebeccaPurple"
        )
    )
    return plot([b, s], layout)
end

## Model Setup

In [ ]:
model_params = Dict([
    (("delta_1", ""), 0.01),
    (("delta_2", ""), 10.0),
    (("delta_3", ""), 20.0),
    #
    (("chi_1", ""), 0.01),
    (("chi_2", ""), 0.10),
    (("chi_3", ""), 0.15),
    #
    (("EUR_f_1", ""), 0.0070),
    (("EUR_f_2", ""), 0.0070),
    (("EUR_f_3", ""), 0.0070),
    #
    (("USD_f_1", ""), 0.0085),
    (("USD_f_2", ""), 0.0085),
    (("USD_f_3", ""), 0.0085),
    #
    (("GBP_f_1", ""), 0.0085),
    (("GBP_f_2", ""), 0.0085),
    (("GBP_f_3", ""), 0.0085),
    #
    (("USD-EUR_x", ""), 0.090),
    (("GBP-EUR_x", ""), 0.075),
    #
    (("EUR_f_1", "EUR_f_2"), 0.1),
    (("EUR_f_2", "EUR_f_3"), 0.2),
    (("EUR_f_1", "EUR_f_3"), 0.2),
    #
    (("USD_f_1", "USD_f_2"), 0.1),
    (("USD_f_2", "USD_f_3"), 0.2),
    (("USD_f_1", "USD_f_3"), 0.2),
    #
    (("GBP_f_1", "GBP_f_2"), 0.1),
    (("GBP_f_2", "GBP_f_3"), 0.2),
    (("GBP_f_1", "GBP_f_3"), 0.2),
    #
    (("EUR_f_1", "USD-EUR_x"), 0.15),
    (("EUR_f_2", "USD-EUR_x"), 0.15),
    (("EUR_f_3", "USD-EUR_x"), 0.15),
    #
    (("USD_f_1", "USD-EUR_x"), -0.10),
    (("USD_f_2", "USD-EUR_x"), -0.10),
    (("USD_f_3", "USD-EUR_x"), -0.10),
    #
    (("GBP_f_1", "USD-EUR_x"), 0.00),
    (("GBP_f_2", "USD-EUR_x"), 0.00),
    (("GBP_f_3", "USD-EUR_x"), 0.00),
    #
    (("EUR_f_1", "GBP-EUR_x"), 0.05),
    (("EUR_f_2", "GBP-EUR_x"), 0.05),
    (("EUR_f_3", "GBP-EUR_x"), 0.05),
    #
    (("USD_f_1", "GBP-EUR_x"), -0.20),
    (("USD_f_2", "GBP-EUR_x"), -0.20),
    (("USD_f_3", "GBP-EUR_x"), -0.20),
    #
    (("GBP_f_1", "GBP-EUR_x"), -0.20),
    (("GBP_f_2", "GBP-EUR_x"), -0.20),
    (("GBP_f_3", "GBP-EUR_x"), -0.20),
    #
    (("USD-EUR_x", "GBP-EUR_x"), 0.35),
    #
    (("EUR_f_1", "USD_f_1"), 0.6),
    (("EUR_f_1", "USD_f_2"), 0.2),
    (("EUR_f_1", "USD_f_3"), 0.2),
    #
    (("EUR_f_2", "USD_f_1"), 0.2),
    (("EUR_f_2", "USD_f_2"), 0.6),
    (("EUR_f_2", "USD_f_3"), 0.3),
    #
    (("EUR_f_3", "USD_f_1"), 0.2),
    (("EUR_f_3", "USD_f_2"), 0.3),
    (("EUR_f_3", "USD_f_3"), 0.7),
    #
    (("EUR_f_1", "GBP_f_1"), 0.65),
    (("EUR_f_1", "GBP_f_2"), 0.25),
    (("EUR_f_1", "GBP_f_3"), 0.25),
    #
    (("EUR_f_2", "GBP_f_1"), 0.25),
    (("EUR_f_2", "GBP_f_2"), 0.65),
    (("EUR_f_2", "GBP_f_3"), 0.35),
    #
    (("EUR_f_3", "GBP_f_1"), 0.65),
    (("EUR_f_3", "GBP_f_2"), 0.35),
    (("EUR_f_3", "GBP_f_3"), 0.75),
    #
    (("USD_f_1", "GBP_f_1"), 0.6),
    (("USD_f_1", "GBP_f_2"), 0.2),
    (("USD_f_1", "GBP_f_3"), 0.2),
    #
    (("USD_f_2", "GBP_f_1"), 0.2),
    (("USD_f_2", "GBP_f_2"), 0.6),
    (("USD_f_2", "GBP_f_3"), 0.3),
    #
    (("USD_f_3", "GBP_f_1"), 0.2),
    (("USD_f_3", "GBP_f_2"), 0.3),
    (("USD_f_3", "GBP_f_3"), 0.7),    
]);

In [ ]:
function model(model_params)
    #
    ch = DiffFusion.correlation_holder("Std")
    for t in model_params
        if t[1][2] != ""
            DiffFusion.set_correlation!(ch, t[1][1], t[1][2], t[2])
        end
    end
    #
    t0 = [ 0.0 ]
    d = model_params  # abbreviation
    #
    delta = [ d[("delta_1","")], d[("delta_2","")], d[("delta_3","")] ]
    chi = [ d[("chi_1","")], d[("chi_2","")], d[("chi_3","")] ]
    #
    hjm_eur = DiffFusion.gaussian_hjm_model(
        "EUR",
        DiffFusion.flat_parameter(delta),
        DiffFusion.flat_parameter(chi),
        DiffFusion.backward_flat_volatility("",t0, [ d[("EUR_f_1","")] d[("EUR_f_2","")] d[("EUR_f_3","")] ]' ),  # sigma
        ch,
        nothing,
    )
    fx_usd_eur = DiffFusion.lognormal_asset_model(
        "USD-EUR",
        DiffFusion.flat_volatility("", d[("USD-EUR_x","")]),
        ch,
        nothing,
    )
    hjm_usd = DiffFusion.gaussian_hjm_model(
        "USD",
        DiffFusion.flat_parameter(delta),
        DiffFusion.flat_parameter(chi),
        DiffFusion.backward_flat_volatility("",t0, [ d[("USD_f_1","")] d[("USD_f_2","")] d[("USD_f_3","")] ]' ),  # sigma
        ch,
        fx_usd_eur,
    )
    fx_gbp_eur = DiffFusion.lognormal_asset_model(
        "GBP-EUR",
        DiffFusion.flat_volatility("", d[("GBP-EUR_x","")]),
        ch,
        nothing,
    )
    hjm_gbp = DiffFusion.gaussian_hjm_model(
        "GBP",
        DiffFusion.flat_parameter(delta),
        DiffFusion.flat_parameter(chi),
        DiffFusion.backward_flat_volatility("",t0, [ d[("GBP_f_1","")] d[("GBP_f_2","")] d[("GBP_f_3","")] ]' ),  # sigma
        ch,
        fx_gbp_eur,
    )
    #
    models = [ hjm_eur, fx_usd_eur, hjm_usd, fx_gbp_eur, hjm_gbp ]
    mdl = DiffFusion.simple_model("Std", models)
    #
    empty_key = DiffFusion._empty_context_key
    #
    ctx = DiffFusion.Context("Std",
        DiffFusion.NumeraireEntry("EUR", "EUR", Dict()),
        Dict{String, DiffFusion.RatesEntry}([
            ("EUR", DiffFusion.RatesEntry("EUR","EUR", Dict())),
            ("USD", DiffFusion.RatesEntry("USD", "USD", Dict())),
            ("GBP", DiffFusion.RatesEntry("GBP", "GBP", Dict())),
        ]),
        Dict{String, DiffFusion.AssetEntry}([
            ("USD-EUR", DiffFusion.AssetEntry("USD-EUR", "USD-EUR", "EUR", "USD", "pa/USD-EUR", Dict(), Dict())),
            ("GBP-EUR", DiffFusion.AssetEntry("GBP-EUR", "GBP-EUR", "EUR", "GBP", "pa/GBP-EUR", Dict(), Dict())),
        ]),
        Dict{String, DiffFusion.ForwardIndexEntry}(),
        Dict{String, DiffFusion.FutureIndexEntry}(),
        Dict{String, DiffFusion.FixingEntry}(),
    )
    #
    return (ctx, mdl, ch)
end

In [ ]:
function model_correlation(index_list, keys_and_terms, corr_matrix)
    return [
        corr_matrix[
            findall(x->x==(l[1],l[2]),keys_and_terms)[begin],
            findall(x->x==(l[3],l[4]),keys_and_terms)[begin]
        ]
        for l in index_list
    ]
end

## Update and Output

In [ ]:
function update_plots!(
    param_list, # ::AbstractArray{<:Tuple},
    model_params,
    std_table,
    corr_table;
    plot_vols = false,
    plot_rates_corrs = false,
    plot_fx_corrs = false,
    plot_fx_rates_corrs = false,
    plot_rates_rates_corrs = false,
    )
    #
    for t in param_list
        model_params[(t[1], t[2])] = t[3]
    end
    #
    (ctx, mdl, ch) = model(model_params)
    keys_and_terms = [
        ("EUR", 1),
        ("EUR", 2),
        ("EUR", 5),
        ("EUR", 10),
        ("EUR", 20),
        #
        ("USD", 1),
        ("USD", 2),
        ("USD", 5),
        ("USD", 10),
        ("USD", 20),
        #
        ("GBP", 1),
        ("GBP", 2),
        ("GBP", 5),
        ("GBP", 10),
        ("GBP", 20),
        #
        ("USD-EUR", 0),
        ("GBP-EUR", 0),
    ]
    (v, C) = DiffFusion.reference_rate_volatility_and_correlation(keys_and_terms, ctx, mdl, ch, 0.0, 1.0/12.0)
    #
    model_terms = [1.0, 2.0, 5.0, 10.0, 20.0]
    v_eur = v[1:5]
    v_usd = v[6:10]
    v_gbp = v[11:15]
    v_usd_eur = v[16:16]
    v_gbp_eur = v[17:17];
    #
    if plot_vols
        display("text/markdown", "## Volatilities:")
        display(plot_volatility(std_table, "EUR", model_terms, v_eur, 1.0e+4))
        display(plot_volatility(std_table, "USD", model_terms, v_usd, 1.0e+4))
        display(plot_volatility(std_table, "GBP", model_terms, v_gbp, 1.0e+4))
        display(plot_volatility(std_table, "USD-EUR", [0.0], v_usd_eur, 1.0e+2))
        display(plot_volatility(std_table, "GBP-EUR", [0.0], v_gbp_eur, 1.0e+2))
    end
    #
    rf_eur = [ ("EUR", 1), ("EUR", 2), ("EUR", 5), ("EUR", 10), ("EUR", 20),]
    rf_usd = [ ("USD", 1), ("USD", 2), ("USD", 5), ("USD", 10), ("USD", 20),]
    rf_gbp = [ ("GBP", 1), ("GBP", 2), ("GBP", 5), ("GBP", 10), ("GBP", 20),]
    rf_usd_eur = [ ("USD-EUR", 0),]
    rf_gbp_eur = [ ("GBP-EUR", 0),];
    #
    if plot_rates_corrs
        display("text/markdown", "## Rates Correlations:")
        idx = make_index_list(rf_eur)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
        idx = make_index_list(rf_usd)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
        idx = make_index_list(rf_gbp)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
        end
    #
    if plot_fx_corrs
        display("text/markdown", "## FX Correlations:")
        idx = make_index_list(rf_gbp_eur, rf_usd_eur)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
    end
    #
    if plot_fx_rates_corrs
        display("text/markdown", "## FX versus Rates Correlations:")
        idx = make_index_list(rf_eur, rf_gbp_eur)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
        idx = make_index_list(rf_gbp, rf_gbp_eur)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
        idx = make_index_list(rf_gbp_eur, rf_usd)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
        idx = make_index_list(rf_eur, rf_usd_eur)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
        idx = make_index_list(rf_usd, rf_usd_eur)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))    
        idx = make_index_list(rf_gbp, rf_usd_eur)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
    end
    #
    if plot_rates_rates_corrs
        display("text/markdown", "## Rates versus Rates Correlations:")
        idx = make_index_list(rf_eur, rf_usd)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))    
        idx = make_index_list(rf_eur, rf_gbp)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))    
        idx = make_index_list(rf_gbp, rf_usd)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
    end
end

## Model Analysis

In [ ]:
update_plots!(
    [
    ],
    model_params, std_table, corr_table,
    plot_vols = true,
    plot_rates_corrs = true,
    plot_fx_corrs = true,
    plot_fx_rates_corrs = true,
    plot_rates_rates_corrs = true,
)